In [63]:
import requests
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

import math


In [64]:
llama_pools = {
    'arb_usdc' : 'd9fa8e14-0447-4207-9ae8-7810199dfa1f',
    'arb_usdt' : '3a6cc030-738d-4e19-8a40-e63e9c4d5a6f',
    'arb_usdce' : '7aab7b0f-01c1-4467-bc0d-77826d870f19',
    'arb_dai' : 'a8e3d841-2788-4647-ad54-5a36fac451b1',    
    'pol_usdc' : '1b8b4cdb-0728-42a8-bf13-2c8fea7427ee',
    'pol_usdt' : '7e7821a2-3d20-4ae7-9c3d-04cd57904555',
    'pol_dai' : 'c57bdc97-3100-41ff-845f-075363f6f5a4',
    'pol_usdce' : '37b04faa-95bb-4ccb-9c4e-c70fa167342b',
}

In [65]:
# Initialize empty DataFrames for time series and statistics
time_series_df = pd.DataFrame()
stats_list = []

# Loop through each pool and fetch the data
for pool_name, pool_id in llama_pools.items():
    # Define the API URL for the current pool
    api_url = f"https://yields.llama.fi/chartlendBorrow/{pool_id}"
    
    # Fetch the data from the API
    response = requests.get(api_url)
    
    if response.status_code == 200:
        data = response.json()['data']
        
        # Convert the list of dictionaries into a pandas DataFrame
        pool_df = pd.DataFrame(data)
        
        # Ensure the timestamp is in datetime format
        pool_df['timestamp'] = pd.to_datetime(pool_df['timestamp'])
        
        # Set the timestamp as the index
        pool_df.set_index('timestamp', inplace=True)
        
        # Rename columns to include pool name for clarity
        pool_df.rename(columns={'apyBase': f'{pool_name}_apyBase', 'apyBaseBorrow': f'{pool_name}_apyBaseBorrow'}, inplace=True)
        
        # Merge with the main time series DataFrame
        if time_series_df.empty:
            time_series_df = pool_df[[f'{pool_name}_apyBase', f'{pool_name}_apyBaseBorrow']]
        else:
            time_series_df = time_series_df.join(pool_df[[f'{pool_name}_apyBase', f'{pool_name}_apyBaseBorrow']], how='outer')
        
        # Compute statistics for apyBase
        apyBase_stats = {
            'Pool': pool_name,
            'Metric': 'apyBase',
            'Average': pool_df[f'{pool_name}_apyBase'].mean(),
            'Median': pool_df[f'{pool_name}_apyBase'].median(),
            'Volatility': pool_df[f'{pool_name}_apyBase'].std(),
            'Max': pool_df[f'{pool_name}_apyBase'].max(),
            'Min': pool_df[f'{pool_name}_apyBase'].min(),
            '10th Percentile': pool_df[f'{pool_name}_apyBase'].quantile(0.1),
            '90th Percentile': pool_df[f'{pool_name}_apyBase'].quantile(0.9),
        }
        stats_list.append(apyBase_stats)
        
        # Compute statistics for apyBaseBorrow
        apyBaseBorrow_stats = {
            'Pool': pool_name,
            'Metric': 'apyBaseBorrow',
            'Average': pool_df[f'{pool_name}_apyBaseBorrow'].mean(),
            'Median': pool_df[f'{pool_name}_apyBaseBorrow'].median(),
            'Volatility': pool_df[f'{pool_name}_apyBaseBorrow'].std(),
            'Max': pool_df[f'{pool_name}_apyBaseBorrow'].max(),
            'Min': pool_df[f'{pool_name}_apyBaseBorrow'].min(),
            '10th Percentile': pool_df[f'{pool_name}_apyBaseBorrow'].quantile(0.1),
            '90th Percentile': pool_df[f'{pool_name}_apyBaseBorrow'].quantile(0.9),
        }
        stats_list.append(apyBaseBorrow_stats)
    else:
        print(f"Error: Unable to fetch data for {pool_name} (status code: {response.status_code})")

# Convert the stats list to a DataFrame
stats_df = pd.DataFrame(stats_list)


In [66]:
# Categorize pools into Polygon and Arbitrum
stats_df['Category'] = stats_df['Pool'].apply(lambda x: 'Polygon' if x.startswith('pol') else 'Arbitrum')

# Select numeric columns for aggregation
numeric_columns = ['Average', 'Median', 'Volatility', 'Max', 'Min', '10th Percentile', '90th Percentile']

# Calculate average metrics by category
average_metrics_df = stats_df.groupby('Category')[numeric_columns].mean().reset_index()

# Add a row for the 'Metric' column with static values
metric_row = pd.DataFrame({
    'Category': ['apyBase', 'apyBaseBorrow'],
    'Average': [stats_df[stats_df['Metric'] == 'apyBase']['Average'].mean(), stats_df[stats_df['Metric'] == 'apyBaseBorrow']['Average'].mean()],
    'Median': [stats_df[stats_df['Metric'] == 'apyBase']['Median'].median(), stats_df[stats_df['Metric'] == 'apyBaseBorrow']['Median'].median()],
    'Volatility': [stats_df[stats_df['Metric'] == 'apyBase']['Volatility'].mean(), stats_df[stats_df['Metric'] == 'apyBaseBorrow']['Volatility'].mean()],
    'Max': [stats_df[stats_df['Metric'] == 'apyBase']['Max'].max(), stats_df[stats_df['Metric'] == 'apyBaseBorrow']['Max'].max()],
    'Min': [stats_df[stats_df['Metric'] == 'apyBase']['Min'].min(), stats_df[stats_df['Metric'] == 'apyBaseBorrow']['Min'].min()],
    '10th Percentile': [stats_df[stats_df['Metric'] == 'apyBase']['10th Percentile'].quantile(0.1), stats_df[stats_df['Metric'] == 'apyBaseBorrow']['10th Percentile'].quantile(0.1)],
    '90th Percentile': [stats_df[stats_df['Metric'] == 'apyBase']['90th Percentile'].quantile(0.9), stats_df[stats_df['Metric'] == 'apyBaseBorrow']['90th Percentile'].quantile(0.9)]
})

# Append the metric rows to the average metrics DataFrame
final_df = pd.concat([average_metrics_df, metric_row], ignore_index=True)

final_df

,Category,Average,Median,Volatility,Max,Min,10th Percentile,90th Percentile
0,Arbitrum,6.239389,4.089515,5.687516,48.693414,0.951928,1.906975,13.355996
1,Polygon,6.635480,4.618804,5.045389,42.084223,1.521761,2.593744,13.809026
2,apyBase,5.332969,2.706015,4.582836,58.440760,0.168480,0.514872,13.876574
3,apyBaseBorrow,7.541900,3.730010,6.150069,76.526450,0.946540,1.712427,17.879768


In [67]:
time_series_df

,arb_usdc_apyBase,arb_usdc_apyBaseBorrow,arb_usdt_apyBase,arb_usdt_apyBaseBorrow,arb_usdce_apyBase,arb_usdce_apyBaseBorrow,arb_dai_apyBase,arb_dai_apyBaseBorrow,pol_usdc_apyBase,pol_usdc_apyBaseBorrow,pol_usdt_apyBase,pol_usdt_apyBaseBorrow,pol_dai_apyBase,pol_dai_apyBaseBorrow,pol_usdce_apyBase,pol_usdce_apyBaseBorrow
timestamp,,,,,,,,,,,,,,,,
2022-08-08 00:00:00+00:00,NaN,NaN,2.06074,NaN,0.52672,NaN,1.36687,NaN,NaN,NaN,2.63472,NaN,0.85173,NaN,0.62811,NaN
2022-08-09 00:00:00+00:00,NaN,NaN,1.79628,NaN,0.53133,NaN,2.15639,NaN,NaN,NaN,2.51822,NaN,0.85393,NaN,0.54653,NaN
2022-08-10 00:00:00+00:00,NaN,NaN,1.82467,NaN,0.52661,NaN,2.49296,NaN,NaN,NaN,2.47140,NaN,0.90513,NaN,0.57561,NaN
2022-08-11 00:00:00+00:00,NaN,NaN,1.44976,NaN,0.49597,NaN,1.41182,NaN,NaN,NaN,2.66292,NaN,1.03180,NaN,0.53375,NaN
2022-08-12 00:00:00+00:00,NaN,NaN,1.44642,NaN,0.47260,NaN,1.58077,NaN,NaN,NaN,2.67416,NaN,1.02385,NaN,0.52667,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-03 23:01:00.726000+00:00,11.10572,13.59508,10.73798,13.46890,7.94339,16.03975,8.03848,12.19994,13.95413,16.97656,7.05180,8.90851,9.95746,14.76303,13.90191,20.56562
2024-06-04 23:01:17.676000+00:00,10.54498,12.92457,10.26618,12.87353,7.61674,15.28044,5.84886,8.87306,10.04534,12.32574,7.08424,8.92879,5.96149,8.93973,13.11634,19.39760
2024-06-05 23:01:27.264000+00:00,12.78278,15.59091,7.81262,9.79169,9.41116,19.74755,7.21272,10.89444,7.26664,8.98557,7.01692,8.88587,5.97242,8.94782,8.97120,13.20145


In [68]:
# Remove index
time_series_df.reset_index(inplace=True)

# Melt the DataFrame to a long format suitable for Plotly
long_df = time_series_df.melt(id_vars=['timestamp'], var_name='metric', value_name='value')

# Plot the time series data
fig = px.line(long_df, x='timestamp', y='value', color='metric', title='APY Base and Borrow Rates Over Time')

fig.update_layout(
    xaxis_title='Timestamp',
    yaxis_title='APY',
    legend_title='Metric'
)

fig.show()


In [69]:
time_series_df

,timestamp,arb_usdc_apyBase,arb_usdc_apyBaseBorrow,arb_usdt_apyBase,arb_usdt_apyBaseBorrow,arb_usdce_apyBase,arb_usdce_apyBaseBorrow,arb_dai_apyBase,arb_dai_apyBaseBorrow,pol_usdc_apyBase,pol_usdc_apyBaseBorrow,pol_usdt_apyBase,pol_usdt_apyBaseBorrow,pol_dai_apyBase,pol_dai_apyBaseBorrow,pol_usdce_apyBase,pol_usdce_apyBaseBorrow
0,2022-08-08 00:00:00+00:00,NaN,NaN,2.06074,NaN,0.52672,NaN,1.36687,NaN,NaN,NaN,2.63472,NaN,0.85173,NaN,0.62811,NaN
1,2022-08-09 00:00:00+00:00,NaN,NaN,1.79628,NaN,0.53133,NaN,2.15639,NaN,NaN,NaN,2.51822,NaN,0.85393,NaN,0.54653,NaN
2,2022-08-10 00:00:00+00:00,NaN,NaN,1.82467,NaN,0.52661,NaN,2.49296,NaN,NaN,NaN,2.47140,NaN,0.90513,NaN,0.57561,NaN
3,2022-08-11 00:00:00+00:00,NaN,NaN,1.44976,NaN,0.49597,NaN,1.41182,NaN,NaN,NaN,2.66292,NaN,1.03180,NaN,0.53375,NaN
4,2022-08-12 00:00:00+00:00,NaN,NaN,1.44642,NaN,0.47260,NaN,1.58077,NaN,NaN,NaN,2.67416,NaN,1.02385,NaN,0.52667,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,2024-06-03 23:01:00.726000+00:00,11.10572,13.59508,10.73798,13.46890,7.94339,16.03975,8.03848,12.19994,13.95413,16.97656,7.05180,8.90851,9.95746,14.76303,13.90191,20.56562
672,2024-06-04 23:01:17.676000+00:00,10.54498,12.92457,10.26618,12.87353,7.61674,15.28044,5.84886,8.87306,10.04534,12.32574,7.08424,8.92879,5.96149,8.93973,13.11634,19.39760
673,2024-06-05 23:01:27.264000+00:00,12.78278,15.59091,7.81262,9.79169,9.41116,19.74755,7.21272,10.89444,7.26664,8.98557,7.01692,8.88587,5.97242,8.94782,8.97120,13.20145
674,2024-06-06 23:01:21.390000+00:00,13.62373,16.58640,6.98449,8.87224,7.54431,15.09259,8.13447,12.33971,8.36551,10.30293,7.19554,8.99846,25.54788,36.81778,6.65230,9.85687


In [70]:
# Ensure the 'timestamp' column is in datetime format
time_series_df['timestamp'] = pd.to_datetime(time_series_df['timestamp'])

# Split the DataFrame into 'pol' and 'arb' based on column names
pol_df = time_series_df.filter(regex='timestamp|pol_')
arb_df = time_series_df.filter(regex='timestamp|arb_')

# Initialize lists to hold the calculated differences
pol_diff = []
arb_diff = []

# Calculate daily differences for 'pol'
for date, data in pol_df.groupby('timestamp'):
    max_apyBase = data.filter(like='apyBase').max(axis=1).values[0]
    min_apyBaseBorrow = data.filter(like='apyBaseBorrow').min(axis=1).values[0]
    pol_diff.append({'timestamp': date, 'diff': max_apyBase - min_apyBaseBorrow})

# Calculate daily differences for 'arb'
for date, data in arb_df.groupby('timestamp'):
    max_apyBase = data.filter(like='apyBase').max(axis=1).values[0]
    min_apyBaseBorrow = data.filter(like='apyBaseBorrow').min(axis=1).values[0]
    arb_diff.append({'timestamp': date, 'diff': max_apyBase - min_apyBaseBorrow})

# Convert lists to DataFrames
pol_diff_df = pd.DataFrame(pol_diff)
arb_diff_df = pd.DataFrame(arb_diff)

# Merge the DataFrames for plotting
diff_df = pd.merge(pol_diff_df, arb_diff_df, on='timestamp', suffixes=('_pol', '_arb'))

# Plot the time series data
fig = px.line(diff_df, x='timestamp', y=['diff_pol', 'diff_arb'], title='Difference Between Max APY Base and Min APY Base Borrow Over Time')

fig.update_layout(
    xaxis_title='Timestamp',
    yaxis_title='Difference (Max APY Base - Min APY Base Borrow)',
    legend_title='Chain'
)

fig.show()


In [71]:
# Ensure the 'timestamp' column is in datetime format
time_series_df['timestamp'] = pd.to_datetime(time_series_df['timestamp'])

# Initialize lists to hold the calculated differences and max apyBase
overall_diff = []
max_apyBase_series = []

# Calculate daily differences across all assets
for date, data in time_series_df.groupby('timestamp'):
    max_apyBase = data.filter(like='apyBase').max(axis=1).values[0]
    min_apyBaseBorrow = data.filter(like='apyBaseBorrow').min(axis=1).values[0]
    overall_diff.append({'timestamp': date, 'diff': max_apyBase - min_apyBaseBorrow})
    max_apyBase_series.append({'timestamp': date, 'max_apyBase': max_apyBase})

# Convert the lists to DataFrames
overall_diff_df = pd.DataFrame(overall_diff)
max_apyBase_df = pd.DataFrame(max_apyBase_series)

# Merge both DataFrames for plotting
merged_df = pd.merge(overall_diff_df, max_apyBase_df, on='timestamp')

# Plot the time series data
fig = px.line(merged_df, x='timestamp', y=['diff', 'max_apyBase'], title='Difference Between Max APY Base and Min APY Base Borrow & Max APY Base Over Time (Combined Chains)')

fig.update_layout(
    xaxis_title='Timestamp',
    yaxis_title='Values',
    legend_title='Metric'
)

fig.show()


In [72]:
print(pol_diff_df.mean())
print(arb_diff_df.mean())
print(overall_diff_df.mean())

timestamp    2023-07-09 13:52:34.156693760+00:00
diff                                    3.159025
dtype: object
timestamp    2023-07-09 13:52:34.156693760+00:00
diff                                    4.848656
dtype: object
timestamp    2023-07-09 13:52:34.156693760+00:00
diff                                    6.205687
dtype: object


#  Backtesting

In [73]:
LTV = 0.9
initial_collateral = 100  # e.g. $100
stop_condition = 0.8

In [74]:
# Ensure the 'timestamp' column is in datetime format
time_series_df['timestamp'] = pd.to_datetime(time_series_df['timestamp'])

# Initialize lists to hold the calculated differences, max apyBase, and final APY
overall_diff = []
max_apyBase_series = []
final_apy_series = []

number_of_loops = math.ceil(math.log(stop_condition) / math.log(LTV))
total_collateral = initial_collateral * ((1 - LTV**(number_of_loops + 1)) / (1 - LTV))
leverage = total_collateral / initial_collateral

# Filter for specific assets
asset_filter = 'arb'  # Options: 'pol', 'arb', 'both'
if asset_filter == 'pol':
    filtered_df = time_series_df.filter(regex='timestamp|pol_')
elif asset_filter == 'arb':
    filtered_df = time_series_df.filter(regex='timestamp|arb_')
else:
    filtered_df = time_series_df

# Calculate daily differences across all assets and the final APY
for date, data in filtered_df.groupby('timestamp'):
    max_apyBase = data.filter(like='apyBase').max(axis=1).values[0]
    min_apyBaseBorrow = data.filter(like='apyBaseBorrow').min(axis=1).values[0]
    spread = max_apyBase - min_apyBaseBorrow
    
    overall_diff.append({'timestamp': date, 'diff': spread})
    max_apyBase_series.append({'timestamp': date, 'max_apyBase': max_apyBase})
    
    final_apy = (max_apyBase * initial_collateral + (total_collateral - initial_collateral) * spread) / initial_collateral
    final_apy_series.append({'timestamp': date, 'final_apy': final_apy})

# Convert the lists to DataFrames
overall_diff_df = pd.DataFrame(overall_diff)
max_apyBase_df = pd.DataFrame(max_apyBase_series)
final_apy_df = pd.DataFrame(final_apy_series)

# Merge all DataFrames for plotting
merged_df = pd.merge(overall_diff_df, max_apyBase_df, on='timestamp')
merged_df = pd.merge(merged_df, final_apy_df, on='timestamp')

# Plot the time series data
fig = px.line(merged_df, x='timestamp', y=['diff', 'max_apyBase', 'final_apy'], title='Difference between supply and borrow (Filtered by ' + asset_filter.capitalize() + ' chains)')

fig.update_layout(
    xaxis_title='Timestamp',
    yaxis_title='Values',
    legend_title='Metric'
)

fig.show()

In [75]:
# rate on a daily basis
daily_apy = final_apy_df.dropna().assign(final_apy=lambda x: ((1 + x['final_apy']/100)**(1/365)-1))

# Calculate daily growth factors from APY
daily_apy['growth_factor'] = 1 + daily_apy['final_apy']

# Calculate the compounded balance
daily_apy['compounded_balance'] = initial_collateral * daily_apy['growth_factor'].cumprod()

# Plot using Plotly
fig = px.line(daily_apy, x='timestamp', y='compounded_balance', title='Compounded Balance Over Time', labels={'compounded_balance': 'Compounded Balance ($)', 'timestamp': 'Date'})
fig.update_layout(xaxis_title='Date', yaxis_title='Compounded Balance ($)', legend_title='Metric')
fig.show()

In [76]:
daily_apy.set_index("timestamp", inplace=True)
daily_apy.index = pd.to_datetime(daily_apy.index)

# Calculate daily returns
daily_apy['daily_return'] = daily_apy['compounded_balance'].pct_change()

# Calculate annualized return (CAGR)
total_days = (daily_apy.index[-1] - daily_apy.index[0]).days
total_years = total_days / 365.25
ending_balance = daily_apy['compounded_balance'].iloc[-1]
beginning_balance = daily_apy['compounded_balance'].iloc[0]
annualized_return = (ending_balance / beginning_balance) ** (1 / total_years) - 1

# Calculate volatility (annualized standard deviation of daily returns)
daily_volatility = daily_apy['daily_return'].std()
annualized_volatility = daily_volatility * np.sqrt(252)

# Assume a risk-free rate (e.g., 0 for simplicity)
risk_free_rate = 0

# Calculate Sharpe ratio
sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility

# Calculate maximum drawdown
cumulative_returns = (1 + daily_apy['daily_return']).cumprod()
peak = cumulative_returns.cummax()
drawdown = (cumulative_returns - peak) / peak
max_drawdown = drawdown.min()

# Output the metrics
metrics = {
    "Annualized Return": annualized_return * 100,
    "Annualized Volatility": annualized_volatility * 100,
    "Sharpe Ratio": sharpe_ratio,
    "Maximum Drawdown": max_drawdown
}

metrics_df = pd.DataFrame(metrics, index=[0])
metrics_df

,Annualized Return,Annualized Volatility,Sharpe Ratio,Maximum Drawdown
0,19.813705,0.737151,26.878777,0.0


In [78]:
print("--------Average APY--------")
print(final_apy_df[201:].mean())
print(final_apy_df[:200].mean())
print(final_apy_df.mean())

print("")
print("--------Gas cost for $100k capital--------")
print("1h rebalancing")
cost_per_reblancing = 4*number_of_loops
print(cost_per_reblancing, "txs on average per rebalancing")
print("Number of tx per day:", cost_per_reblancing*24)
print("cost per tx:", 0.05, "$")
print("Daily gas cost:", 0.05*cost_per_reblancing*24, "$")
print("Annual gas cost:", 365*0.05*cost_per_reblancing*24, "$")
print("With $100k capital:", (365*0.05*cost_per_reblancing*24/100000)*100,"%")

--------Average APY--------
timestamp    2023-10-16 21:58:29.294235904+00:00
final_apy                              26.263616
dtype: object
timestamp    2022-11-16 07:47:46.122604800+00:00
final_apy                               7.447342
dtype: object
timestamp    2023-07-09 13:52:34.156693760+00:00
final_apy                              21.528623
dtype: object

--------Gas cost for $100k capital--------
1h rebalancing
12 txs on average per rebalancing
Number of tx per day: 288
cost per tx: 0.05 $
Daily gas cost: 14.400000000000002 $
Annual gas cost: 5256.0 $
With $100k capital: 5.256 %
